## GAN 

In [11]:
import tensorflow as tf
import multiprocessing
import os
import numpy as np
import glob
# scipy1.3.0  中没有 scipy.misc.toimage 方法，scipy1.0.0 中有，可以安装指定版本
from scipy.misc import toimage  # scipy.misc 中缺少toimage方法，用PIL中的Image.fromarray代替
from PIL import Image 
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [12]:
config = ConfigProto()   
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')
assert np.__version__.startswith('1.16.2')

/home/kukafee/environments/tf2_py3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1717: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [13]:
# 定义生成器G   向量[b, 100] => 图片[b, 81, 81, 3]
class Generator(keras.Model):
    def __init__(self):
        super(Generator, self).__init__()
        # 输入一个向量，经过生成器，变成一个图片，图片逐渐变大，通道数不断减少
        # z:[b, 100] => [b, 3*3*512]  全连接层        
        self.fc = layers.Dense(3*3*512)
        
        # [b, 3, 3, 512] => [b, 9, 9, 256]   反卷积层(输出通道数_卷积核数量；核的大小；步长；padding)
        # Conv2DTranspose 是考虑什么样的feature_map通过这个卷积层变成了现在的feature_map
        self.conv1 = layers.Conv2DTranspose(256, 3, 3, 'valid')
        self.bn1 = layers.BatchNormalization()
        # [b, 9, 9, 256] => [b, 29, 29, 128]  此处不是[b, 27, 27, 128]可能是因为受卷积核大小的影响吧
        self.conv2 = layers.Conv2DTranspose(128, 5, 3, 'valid')
        self.bn2 = layers.BatchNormalization()
        # [b, 29, 29, 128] => [b, 88, 88, 3]
        self.conv3 = layers.Conv2DTranspose(3, 4, 3, 'valid')
        
        
    def call(self, inputs, training=None):
        # [z, 100] => [z, 3*3*512]
        x = self.fc(inputs)
#         print(x.shape)
        
        # 获得图片feature_map
        x = tf.reshape(x, [-1, 3, 3, 512])        
        # 通过leaky_relu() 函数
        x = tf.nn.leaky_relu(x)
#         print(x.shape)
        
        # [b, 3, 3, 512] => [b, 9, 9, 256]
        x = tf.nn.leaky_relu(self.bn1(self.conv1(x), training=training))
#         print(x.shape)
        
        # [b, 9, 9, 256] => [b, 29, 29, 128]  因为卷积核比较大，是(5*5)所以得到是29*29
        x = tf.nn.leaky_relu(self.bn2(self.conv2(x), training=training))
#         print(x.shape)
        
        # [b, 29, 29, 128] => [b, 88, 88, 3]
        x = self.conv3(x)
        x = tf.tanh(x)
        
        return x    # [b, 88, 88, 3]
    
    
# 定义判别器D  图片[b, 81, 81, 3] => 向量[b, 1]
class Discriminator(keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        # [b, 81, 81, 3] => [b, 27, 27, 64]
        self.conv1 = layers.Conv2D(64, 5, 3, 'valid')
        self.bn1 = layers.BatchNormalization()
        # [b, 27, 27, 64] => [b, 9, 9, 128]
        self.conv2 = layers.Conv2D(128, 5, 3, 'valid')
        self.bn2 = layers.BatchNormalization()
        # [b, 9, 9, 128] => [b, 3, 3, 256]
        self.conv3 = layers.Conv2D(256, 5, 3, 'valid')
        self.bn3 = layers.BatchNormalization()
        
        # [b, 3, 3, 256] => [b, -1]
        # 调用 layers.Flatten() 函数
        self.flatten = layers.Flatten()
        # 通过一个全连接层，输出节点数为1
        self.fc = layers.Dense(1)
        
    def call(self, inputs, training=None):
        x = tf.nn.leaky_relu(self.bn1(self.conv1(inputs), training=training))
        x = tf.nn.leaky_relu(self.bn2(self.conv2(x), training=training))
        x = tf.nn.leaky_relu(self.bn3(self.conv3(x), training=training))
        
        x = self.flatten(x)
        logits = self.fc(x)
        
        return logits    # [b, 1]
    

In [14]:
# # 定义测试函数 
# def main():
#     d_net = Discriminator()
#     g_net = Generator()
    
#     x = tf.random.normal([2, 96, 96, 3])
#     z = tf.random.normal([2, 200])
#     x_prob = d_net(x)
#     x_hat = g_net(z)
#     print(x_prob)
#     print(x_hat.shape)

# # 调用测试函数
# main()

In [15]:
# 保存图片的函数save_result()
# save_result(fake_image.numpy(), 10, img_path, color_mode='P')    # [100, 88, 88, 3] 在 -1 到 1 之间
def save_result(val_out, val_block_size, image_path, color_mode):
    def preprocess(img):
        img = ((img + 1.0) * 127.5).astype(np.uint8)    # 变换到（0， 255），再转换数据类型为np.uint8
        # img = img.astype(np.uint8)
        return img

    preprocesed = preprocess(val_out)    # 获得经过preprocess()处理过的图片数据，在（0,255）之间     
    single_row = np.array([])     # 定义一行
    final_image = np.array([])    # 每次添加 single_row 一行的数据
    for b in range(val_out.shape[0]):    # val_out.shape[0] = 100,一张一张处理图片；b = 0, 1, ... 99
        # concat image into a row
        if single_row.size == 0:
            single_row = preprocesed[b, :, :, :] # preprocessed[0]表示第b张图片，并添加到single_row数组
        else:
            single_row = np.concatenate((single_row, preprocesed[b, :, :, :]), axis=1)

        # concat image row to final_image
        if (b+1) % val_block_size == 0:   # 每val_block_size（此处是10）张截取一行，当b=9时执行
            if final_image.size == 0:    
                final_image = single_row    # 把 single_row 中的数据拷贝至 final_image
            else:    # axis=0 表示 每次添加一行图片数据
                final_image = np.concatenate((final_image, single_row), axis=0)

            # reset single row
            single_row = np.array([])

    if final_image.shape[2] == 1:    # 防止 final_image 出现 [10, 10, 1]情况，squeeze 高维度的1
        final_image = np.squeeze(final_image, axis=2)
    
    toimage(final_image).save(image_path)


In [16]:
# 定义 make_anime_dataset()  用于获取anime数据集中的每一张图片
def make_anime_dataset(img_paths, batch_size, resize=88, drop_remainder=True, shuffle=True, repeat=1):
    @tf.function
    def _map_fn(img):
        img = tf.image.resize(img, [resize, resize])
        img = tf.clip_by_value(img, 0, 255)
        img = img / 127.5 - 1
        return img

    dataset = disk_image_batch_dataset(img_paths,
                                          batch_size,
                                          drop_remainder=drop_remainder,
                                          map_fn=_map_fn,
                                          shuffle=shuffle,
                                          repeat=repeat)
    img_shape = (resize, resize, 3)
    len_dataset = len(img_paths) // batch_size

    return dataset, img_shape, len_dataset


def batch_dataset(dataset,
                  batch_size,
                  drop_remainder=True,
                  n_prefetch_batch=1,
                  filter_fn=None,
                  map_fn=None,
                  n_map_threads=None,
                  filter_after_map=False,
                  shuffle=True,
                  shuffle_buffer_size=None,
                  repeat=None):
    # set defaults
    if n_map_threads is None:
        n_map_threads = multiprocessing.cpu_count()
    if shuffle and shuffle_buffer_size is None:
        shuffle_buffer_size = max(batch_size * 128, 2048)  # set the minimum buffer size as 2048

    # [*] it is efficient to conduct `shuffle` before `map`/`filter` because `map`/`filter` is sometimes costly
    if shuffle:
        dataset = dataset.shuffle(shuffle_buffer_size)

    if not filter_after_map:
        if filter_fn:
            dataset = dataset.filter(filter_fn)

        if map_fn:
            dataset = dataset.map(map_fn, num_parallel_calls=n_map_threads)

    else:  # [*] this is slower
        if map_fn:
            dataset = dataset.map(map_fn, num_parallel_calls=n_map_threads)

        if filter_fn:
            dataset = dataset.filter(filter_fn)

    dataset = dataset.batch(batch_size, drop_remainder=drop_remainder)

    dataset = dataset.repeat(repeat).prefetch(n_prefetch_batch)

    return dataset


def memory_data_batch_dataset(memory_data,
                              batch_size,
                              drop_remainder=True,
                              n_prefetch_batch=1,
                              filter_fn=None,
                              map_fn=None,
                              n_map_threads=None,
                              filter_after_map=False,
                              shuffle=True,
                              shuffle_buffer_size=None,
                              repeat=None):
    """Batch dataset of memory data.

    Parameters
    ----------
    memory_data : nested structure of tensors/ndarrays/lists

    """
    dataset = tf.data.Dataset.from_tensor_slices(memory_data)
    dataset = batch_dataset(dataset,
                            batch_size,
                            drop_remainder=drop_remainder,
                            n_prefetch_batch=n_prefetch_batch,
                            filter_fn=filter_fn,
                            map_fn=map_fn,
                            n_map_threads=n_map_threads,
                            filter_after_map=filter_after_map,
                            shuffle=shuffle,
                            shuffle_buffer_size=shuffle_buffer_size,
                            repeat=repeat)
    return dataset


def disk_image_batch_dataset(img_paths,
                             batch_size,
                             labels=None,
                             drop_remainder=True,
                             n_prefetch_batch=1,
                             filter_fn=None,
                             map_fn=None,
                             n_map_threads=None,
                             filter_after_map=False,
                             shuffle=True,
                             shuffle_buffer_size=None,
                             repeat=None):
    """Batch dataset of disk image for PNG and JPEG.

    Parameters
    ----------
        img_paths : 1d-tensor/ndarray/list of str
        labels : nested structure of tensors/ndarrays/lists

    """
    if labels is None:
        memory_data = img_paths
    else:
        memory_data = (img_paths, labels)

    def parse_fn(path, *label):
        img = tf.io.read_file(path)
        img = tf.image.decode_png(img, 3)  # fix channels to 3
        return (img,) + label

    if map_fn:  # fuse `map_fn` and `parse_fn`
        def map_fn_(*args):
            return map_fn(*parse_fn(*args))
    else:
        map_fn_ = parse_fn

    dataset = memory_data_batch_dataset(memory_data,
                                        batch_size,
                                        drop_remainder=drop_remainder,
                                        n_prefetch_batch=n_prefetch_batch,
                                        filter_fn=filter_fn,
                                        map_fn=map_fn_,
                                        n_map_threads=n_map_threads,
                                        filter_after_map=filter_after_map,
                                        shuffle=shuffle,
                                        shuffle_buffer_size=shuffle_buffer_size,
                                        repeat=repeat)

    return dataset

In [17]:
# 定义 loss_ones()函数，logits 与 [1, 1, 1, ..., 1] 之间的误差
def loss_ones(logits):
    # [b, 1]  VS. [b] = [1, 1, ... , 1]
    loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=tf.ones_like(logits))
    return tf.reduce_mean(loss)

# 定义 loss_zeros()函数，logits 与 [0, 0, 0, ..., 0] 之间的误差
def loss_zeros(logits):
    loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=tf.zeros_like(logits))
    return tf.reduce_mean(loss)

# 定义计算判别器discriminator的损失函数
# d_loss = d_loss_fn(generator, discriminator, batch_z, batch_x, is_training)
def d_loss_fn(generator, discriminator, batch_z, batch_x, is_training):
    # treat real image as real 对于真实的图片数据
    d_real_logits = discriminator(batch_x, is_training)
    # treat generated image as fake 对于生成的图片数据
    d_fake_logits = discriminator(generator(batch_z, is_training), is_training)
    # 计算真实图片的损失
    d_loss_real = loss_ones(d_real_logits)
    # 计算生成图片的损失（fake）
    d_loss_fake = loss_zeros(d_fake_logits)
    loss = d_loss_real + d_loss_fake
    return loss

# 定义计算生成器generator的损失函数
# g_loss = g_loss_fn(generator, discriminator, batch_z, is_training):
def g_loss_fn(generator, discriminator, batch_z, is_training):
    d_fake_logits = discriminator(generator(batch_z, is_training), is_training)
    g_loss_fake = loss_ones(d_fake_logits)    
    return g_loss_fake

In [18]:
# 定义输入到生成器的向量维度
z_dim = 100    # z[b, 100] => x_hat[b, 88, 88, 3] => x_hat_pro[b, 1]
# 定义训练世代数
epochs = 3000000  # 三百万次
# epochs = 200
# 定义批处理数据大小
batch_size = 1024
# 定义学习率
learning_rate = 2e-3
# 定义训练参数
is_training = True
# 定义数据集中图片的路径
#返回所有匹配的文件路径列表。它只有一个参数pathname，定义了文件路径匹配规则
img_path = glob.glob('/home/kukafee/shared/faces/*.jpg')
# 获取数据集中每批次的数据
dataset, img_shape, _ = make_anime_dataset(img_path, batch_size)
# print(dataset, img_shape) # <PrefetchDataset shapes: (512, 88, 88, 3), types: tf.float32> (88, 88, 3)
# 构建迭代器，并查看其中的元素
# sample = next(iter(dataset))
# print(sample.shape, tf.reduce_max(sample).numpy(), tf.reduce_min(sample).numpy())
# 使得可迭代对象可循环
dataset = dataset.repeat()
db_iter = iter(dataset)    # 构建迭代器 db_iter

# 定义并构建生成器网络
generator = Generator()
generator.build(input_shape = (None, z_dim))
# generator.summary()
# 定义并构建判别器网络
discriminator = Discriminator()
discriminator.build(input_shape = (None, 88, 88, 3))
# discriminator.summary()
# 定义优化器 
# 其中参数 beta_1: A float value or a constant float tensor. The exponential decay rate for the 1st
# moment estimates.
g_optimizer = tf.optimizers.Adam(learning_rate=learning_rate, beta_1=0.5)
d_optimizer = tf.optimizers.Adam(learning_rate=learning_rate, beta_1=0.5)


In [ ]:
# 训练并测试GAN
def main():
    # 循环每一个世代
    for epoch in range(epochs):
        # 定义一批要送入生成器的随机均匀分布数据
        # z[b, 100] => x_hat[b, 88, 88, 3] => x_hat_pro[b, 1]
        batch_z = tf.random.uniform([batch_size, z_dim], minval=-1., maxval=1.)
        # 从迭代器 db_iter 中生成要送入判别器的图片数据
        batch_x = next(db_iter)
        for i in range(2):            
            # 训练判别器D——discriminator
            with tf.GradientTape() as tape:
                d_loss = d_loss_fn(generator, discriminator, batch_z, batch_x, is_training)
            # 计算梯度
            grads = tape.gradient(d_loss, discriminator.trainable_variables)
            # 更新参数
            d_optimizer.apply_gradients(zip(grads, discriminator.trainable_variables))
                
        # 训练生成器G——generator
        with tf.GradientTape() as tape:
            g_loss = g_loss_fn(generator, discriminator, batch_z, is_training)
        grads = tape.gradient(g_loss, generator.trainable_variables)
        g_optimizer.apply_gradients(zip(grads, generator.trainable_variables))
        
        
        if epoch % 100 == 0:
            print(epoch, 'd-loss: ', float(d_loss), 'g-loss: ', float(g_loss))
            
            # 测试生气器网络效果
            # 从正态分布中随机产生数据
            z = tf.random.uniform([100, z_dim])    # [100, 100]
            fake_image = generator(z, training=False)    # [100, 88, 88, 3]
            # '/home/kukafee/workspace/picture/GAN/fake_epoch_%d.png'%epoch
            img_path = os.path.join('/home/kukafee/workspace/picture/GAN', 'fake_epoch_%d.png'%epoch)
            save_result(fake_image.numpy(), 10, img_path, color_mode='P')
            
        if epoch % 1000 == 0:            
            # 保存 network 的******参数******到文件
            generator.save_weights('/home/kukafee/workspace/save_model/GAN/G_weight_%d.ckpt'%epoch)
            # 打印信息
            print('Saved G_weights: %d'%epoch)

In [ ]:
main()

0 d-loss:  0.0718667209148407 g-loss:  7.487253665924072


/home/kukafee/environments/tf2_py3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.


Saved G_weights: 0
100 d-loss:  0.6367347240447998 g-loss:  2.079136610031128
200 d-loss:  0.5834982395172119 g-loss:  2.646066665649414
300 d-loss:  0.7264952659606934 g-loss:  2.5978095531463623
400 d-loss:  0.7830707430839539 g-loss:  2.7816150188446045
500 d-loss:  0.5083626508712769 g-loss:  2.8121726512908936
600 d-loss:  0.4349428713321686 g-loss:  3.2510011196136475
700 d-loss:  0.8292799592018127 g-loss:  4.630350112915039
800 d-loss:  0.39947476983070374 g-loss:  4.245325088500977
900 d-loss:  0.4732692837715149 g-loss:  3.1950910091400146
1000 d-loss:  0.22153708338737488 g-loss:  4.5294718742370605
Saved G_weights: 1000
1100 d-loss:  0.3207417130470276 g-loss:  4.110607624053955
1200 d-loss:  0.3428756892681122 g-loss:  4.444893836975098
1300 d-loss:  0.6931564807891846 g-loss:  3.2388126850128174
1400 d-loss:  0.2429182231426239 g-loss:  4.16689395904541
1500 d-loss:  0.2755606174468994 g-loss:  3.3392763137817383
1600 d-loss:  0.47679081559181213 g-loss:  3.06713628768920

13100 d-loss:  0.036004118621349335 g-loss:  6.073843955993652
13200 d-loss:  0.09015777707099915 g-loss:  5.097859859466553
13300 d-loss:  0.06839781999588013 g-loss:  5.648865699768066
13400 d-loss:  0.07355491071939468 g-loss:  5.790332317352295
13500 d-loss:  0.0756712257862091 g-loss:  5.076382160186768
13600 d-loss:  0.02178790420293808 g-loss:  6.775018692016602
13700 d-loss:  0.18581831455230713 g-loss:  5.417026996612549
13800 d-loss:  0.09198113530874252 g-loss:  5.086434364318848
13900 d-loss:  0.12090808153152466 g-loss:  5.623741626739502
14000 d-loss:  0.14621379971504211 g-loss:  4.867206573486328
Saved G_weights: 14000
14100 d-loss:  0.07356808334589005 g-loss:  5.450902462005615
14200 d-loss:  1.3403098583221436 g-loss:  4.235217094421387
14300 d-loss:  0.06046878546476364 g-loss:  5.802402973175049
14400 d-loss:  0.021769458428025246 g-loss:  6.602114200592041
14500 d-loss:  0.07790430635213852 g-loss:  5.629189968109131
14600 d-loss:  0.07975228875875473 g-loss:  5.8

25900 d-loss:  0.06096625328063965 g-loss:  6.51792049407959
26000 d-loss:  0.03932839632034302 g-loss:  6.730444431304932
Saved G_weights: 26000
26100 d-loss:  0.09954620897769928 g-loss:  6.668071746826172
26200 d-loss:  0.043621040880680084 g-loss:  8.662736892700195
26300 d-loss:  0.2509566843509674 g-loss:  3.4179444313049316
26400 d-loss:  0.135908842086792 g-loss:  6.940043926239014
26500 d-loss:  0.1970258355140686 g-loss:  4.284502983093262
26600 d-loss:  0.04831460490822792 g-loss:  6.7554707527160645
26700 d-loss:  0.006254047155380249 g-loss:  9.270856857299805
26800 d-loss:  0.022872203961014748 g-loss:  7.436927318572998
26900 d-loss:  0.043473079800605774 g-loss:  7.588587284088135
27000 d-loss:  0.9431150555610657 g-loss:  0.7978678941726685
Saved G_weights: 27000
27100 d-loss:  0.20676526427268982 g-loss:  4.08985710144043
27200 d-loss:  0.011709030717611313 g-loss:  8.794455528259277
27300 d-loss:  0.04156637191772461 g-loss:  6.9836835861206055
27400 d-loss:  0.05255

38600 d-loss:  0.006598150357604027 g-loss:  55.96088409423828
38700 d-loss:  0.009899331256747246 g-loss:  75.1407241821289
38800 d-loss:  0.012187689542770386 g-loss:  52.62348175048828
38900 d-loss:  0.008153031580150127 g-loss:  23.02196502685547
39000 d-loss:  0.003551672911271453 g-loss:  91.9942398071289
Saved G_weights: 39000
39100 d-loss:  0.0020620382856577635 g-loss:  51.28694152832031
39200 d-loss:  0.14496588706970215 g-loss:  112.18732452392578
39300 d-loss:  0.007240184582769871 g-loss:  38.30497741699219
39400 d-loss:  0.00659925676882267 g-loss:  25.879751205444336
39500 d-loss:  0.017151718959212303 g-loss:  51.992088317871094
39600 d-loss:  0.0001442656066501513 g-loss:  17.09652328491211
39700 d-loss:  0.001181091065518558 g-loss:  34.69975280761719
39800 d-loss:  0.00307950540445745 g-loss:  14.423110961914062
39900 d-loss:  0.0002196468849433586 g-loss:  26.975914001464844
40000 d-loss:  0.000203285992029123 g-loss:  22.316749572753906
Saved G_weights: 40000
40100